In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import hvplot.pandas
import panel as pn

pn.extension()

data = pd.read_csv('https://tinyurl.com/ChrisCoDV/Products/DailySales.csv', index_col=0)
data.index = pd.to_datetime(data.index)
print(data.head())


In [ ]:
def get_correlation_plots(product_x, product_y, date_range):
    df = data.loc[pd.to_datetime(date_range[0]) : pd.to_datetime(date_range[1])] # restrict to date range
    return df.hvplot.scatter(frame_height=300, frame_width=300,
                             x=product_x, y=product_y,
                             title=product_x + ' vs ' + product_y,
                             size=10, padding=0.1) +\
           df.corr().hvplot.heatmap(frame_height=300, frame_width=300,
                                    title='Product correlations',
                                    rot=90, cmap='coolwarm'
                                   ).opts(invert_yaxis=True, clim=(-1, 1))

In [ ]:
correlation_title = '## Correlations'

product_x = pn.widgets.Select(name='x-axis', options=list(data.columns), width=200)
product_y = pn.widgets.Select(name='y-axis', options=list(data.columns), width=200)
corr_date_range = pn.widgets.DateRangeSlider(name='Dates',
                                        value=(data.index[0], data.index[-1]),
                                        start=data.index[0], end=data.index[-1],
                                        width=200)

correlation_row = pn.Row(
    pn.Column(correlation_title, product_x, product_y, corr_date_range),
    get_correlation_plots(product_x.value, product_y.value, corr_date_range.value)
)

def update_correlation_row(event):
    correlation_row[1].object = get_correlation_plots(product_x.value, product_y.value, corr_date_range.value)

product_x.param.watch(update_correlation_row, 'value')
product_y.param.watch(update_correlation_row, 'value')
corr_date_range.param.watch(update_correlation_row, 'value')

correlation_row.servable()